In [1]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
#Please add the folder name of the dataset to run it on different dataset.
dataset = 'AWA2'
path = 'E:/Sushree/Dataset/data/xlsa17/data/'

res101 = scipy.io.loadmat(path + dataset + '/res101.mat')
att_splits = scipy.io.loadmat(path + dataset + '/att_splits.mat')

In [14]:
signature = att_splits['att']
#signature = att_splits['original_att']
#signature = signature/100
print(signature.shape) #(85, 50)

attribute = signature.transpose()
attribute[attribute<0] = 0
print(attribute, attribute.shape)#(50, 85)

(85, 50)
[[0.     0.     0.     ... 0.0235 0.097  0.0838]
 [0.3925 0.0139 0.     ... 0.5864 0.2014 0.1139]
 [0.834  0.6479 0.     ... 0.1577 0.1341 0.1542]
 ...
 [0.6357 0.431  0.     ... 0.3595 0.2826 0.05  ]
 [0.5531 0.5546 0.     ... 0.0504 0.1889 0.7299]
 [0.1022 0.2153 0.2773 ... 0.0396 0.1405 0.3798]] (50, 85)


In [4]:
import gensim.downloader as api
import pandas as pd
print('Load pretrain w2v model')

#model_name = 'word2vec-google-news-300' # length = 300
#model = api.load(model_name)

import transformers
from transformers import BertTokenizer, TFBertModel, BartTokenizer, TFBartModel, GPT2Tokenizer, TFGPT2Model
from transformers import OpenAIGPTTokenizer, TFOpenAIGPTModel

# Bert model
#---------------------------------------------------------------------------
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # length 768
#model = TFBertModel.from_pretrained("bert-base-uncased")

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased') # 1024
model = TFBertModel.from_pretrained("bert-large-uncased")

# Bart model
#---------------------------------------------------------------------------
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-base') # length 768
#model = TFBartModel.from_pretrained("facebook/bart-base")

#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large') # length 1024
#model = TFBartModel.from_pretrained("facebook/bart-large")

# GPT2 model
#---------------------------------------------------------------------------
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # length 768
#model = TFGPT2Model.from_pretrained('gpt2')

# GPT model
#---------------------------------------------------------------------------
#tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')# length 768
#model = TFOpenAIGPTModel.from_pretrained('openai-gpt')

Load pretrain w2v model


C:\Users\Admin\anaconda3\envs\en3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassifica

In [5]:
#tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
#model = TFOpenAIGPTModel.from_pretrained('openai-gpt')
#encoded_input = tokenizer('sushree', return_tensors='tf')
#print(encoded_input)
#length = encoded_input.input_ids.shape[1]
#print(length)

In [6]:
#print(tokenizer)
#out = model(encoded_input).last_hidden_state[:, 2, :]
#print(out.shape)

In [7]:

dim_w2v = 1024

#%%
replace_word = [('newworld','new world'),('oldworld','old world'),('nestspot','nest spot'),('toughskin','tough skin'),
                ('longleg','long leg'), ('longneck', 'long neck'), ('chewteeth','chew teeth'),('meatteeth','meat teeth'),('strainteeth','strain teeth'),
                ('quadrapedal','quadrupedal')]  # for AWA2


#For AWA2
path = 'E:/Sushree/Dataset/Animals_with_Attributes2/attribute/predicates.txt'
df=pd.read_csv(path,sep='\t',header = None, names = ['idx','des'])
des = df['des'].values

#%% replace out of dictionary (OOD) words
for pair in replace_word:
    for idx,s in enumerate(des):
        des[idx] = s.replace(pair[0],pair[1])
print('Done replacing OOD words')

df['new_des'] = des
df.to_csv('E:/Sushree/Dataset/Animals_with_Attributes2/attribute/new_des.csv')
print('Done preprocessing attribute des')

import pickle


# for word2vec
# -----------------------------------------------------------------------------------
#counter_err = 0
#
#all_w2v = []
#for s in des:
##    print(s)
#    words = s.split(' ')
#    if words[-1] == '':     #remove empty element
#        words = words[:-1]
#    w2v = np.zeros(dim_w2v)
#    for w in words:
#        try:
#            w2v += model[w]
#        except Exception as e:
#            print(e)
#            counter_err += 1
#    w2v = w2v / len(words)  
#    all_w2v.append(w2v[np.newaxis,:])
#    
#print('counter_err ',counter_err)

#w2v_att = np.concatenate(all_w2v,axis=0)
#print(w2v_att, w2v_att.shape)

# for Bert attributes
# -----------------------------------------------------------------------------------
counter = 0
w2v_att = np.zeros((signature.shape[0], dim_w2v))
for s in des:
    print(s)
    w2v = np.zeros(dim_w2v)
    encoded_input = tokenizer(s, return_tensors='tf')
    length = encoded_input.input_ids.shape[1]
    #print(length)
    for i in range(length-2): # for Bert, Bart
    #for i in range(length): # for GPT2
        w2v = w2v + model(encoded_input).last_hidden_state[:, i+1, :] # for Bert, Bart
        #w2v = w2v + model(encoded_input).last_hidden_state[:, i, :] # for GPT2
        #print(model(encoded_input).last_hidden_state[:, i+1, :][:,1:2])
        #print(w2v[:, 1:2])
        #print(w2v.shape)

    w2v = w2v / (length - 2)# for Bert, Bart
    #w2v = w2v / length # for GPT2
    #print(w2v[:, 1:2])
    w2v_att[counter] = w2v
    counter = counter + 1

print('counter ',counter)
print(w2v_att, w2v_att.shape)
# -----------------------------------------------------------------------------------


attribute_new = np.einsum('ij,jl->ijl', attribute, w2v_att)

attribute_new = np.reshape(attribute_new, [attribute_new.shape[0], attribute_new.shape[1]* attribute_new.shape[2]])
print(attribute_new, attribute_new.shape)

#attribute_new[attribute_new<0] = 0
#print(attribute_new, attribute_new.shape)

Done replacing OOD words
Done preprocessing attribute des
black
white
blue
brown
gray
orange
red
yellow
patches
spots
stripes
furry
hairless
tough skin
big
small
bulbous
lean
flippers
hands
hooves
pads
paws
long leg
long neck
tail
chew teeth
meat teeth
buckteeth
strain teeth
horns
claws
tusks
smelly
flys
hops
swims
tunnels
walks
fast
slow
strong
weak
muscle
bipedal
quadrupedal
active
inactive
nocturnal
hibernate
agility
fish
meat
plankton
vegetation
insects
forager
grazer
hunter
scavenger
skimmer
stalker
new world
old world
arctic
coastal
desert
bush
plains
forest
fields
jungle
mountains
ocean
ground
water
tree
cave
fierce
timid
smart
group
solitary
nest spot
domestic
counter  85
[[-0.09144888 -0.13553846 -0.12541848 ... -0.02989655 -0.38680694
   0.24406666]
 [-0.29638147  0.14530022  0.20919298 ... -0.13767773 -0.49952391
  -0.02497395]
 [-0.17159468  0.01720696 -0.01027012 ...  0.11106262  0.08405223
   0.08059736]
 ...
 [-0.36489192  0.06430094 -0.17526677 ...  0.12331312 -0.369820

In [8]:

# total number of instances or images = 37322: ranges from 0 to 37321

trainval_loc = np.squeeze(att_splits['trainval_loc']-1) # -1: to consider the overflow problem
print(np.unique(trainval_loc), np.max(np.unique(trainval_loc))) # smallest location: 2, largest location 37321

labels = res101['labels']# direct class labels
print('labels', labels, labels.shape)# 37322 x 1

print('unique_labels', np.unique(labels), np.unique(labels).shape)# class labels range from 1 to 50, 50 classes

# get the labels for trainval, test seen and test unseen sets

labels_trainval = labels[trainval_loc]
print('labels_trainval', labels_trainval, labels_trainval.shape)

unique_labels_trainval = np.unique(labels_trainval) # labels min:1 max:49
print('unique_labels_trainval', unique_labels_trainval, unique_labels_trainval.shape)# 40 classes



[    2     4     5 ... 37318 37320 37321] 37321
labels [[ 1]
 [ 1]
 [ 1]
 ...
 [38]
 [38]
 [38]] (37322, 1)
unique_labels [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50] (50,)
labels_trainval [[43]
 [22]
 [43]
 ...
 [40]
 [19]
 [46]] (23527, 1)
unique_labels_trainval [ 1  2  3  4  5  6  8 10 11 12 13 14 15 16 17 18 19 20 21 22 25 26 27 28
 29 32 33 35 36 37 38 39 40 42 43 44 45 46 48 49] (40,)


In [9]:
X_features = res101['features']

# locations are already subtracted by 1, so they range from 0 to 37321
trainval_vec = X_features[:, trainval_loc].transpose()
print("Features for trainval:", trainval_vec.shape) #(23527, 2048)

# attribute is defined for all 50 classes, so we cant use locations directly, instead we have to use labels 
# that range from 1 to 50, so we have to subtract 1

trainval_attributes = np.zeros((len(trainval_loc), attribute_new.shape[1]))
for i in range(len(trainval_loc)):
    trainval_attributes[i] = attribute_new[int(labels_trainval[i])-1]

print(trainval_attributes, trainval_attributes.shape)# (23527, 85)


Features for trainval: (23527, 2048)
[[-0.00052664 -0.00078054 -0.00072226 ... -0.00223581 -0.00665391
   0.00469815]
 [ 0.          0.          0.         ... -0.00277371 -0.00825473
   0.00582844]
 [-0.00052664 -0.00078054 -0.00072226 ... -0.00223581 -0.00665391
   0.00469815]
 ...
 [ 0.          0.          0.         ... -0.0064743  -0.01926791
   0.01360456]
 [-0.00076979 -0.00114092 -0.00105574 ... -0.00450891 -0.01341877
   0.00947463]
 [-0.00354577 -0.00525526 -0.00486288 ... -0.05893629 -0.17539784
   0.12384375]] (23527, 87040)


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad
import keras.backend as K

attribute_shape_new = attribute_new.shape[1]
print(attribute_shape_new)

# define model for attribute transformation

inputt = Input(shape = attribute_shape_new)
hidden = Dense(4096, name="layer1", activation='linear')(inputt)
output = Dense(2048, name="layer3", activation='linear')(hidden)

model_transform_attribute = Model(inputs = inputt, outputs = output)

#sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

model_transform_attribute.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

for layer in model_transform_attribute.layers:
        layer.trainable = False
        
model_transform_attribute.summary()

attribute_new = model_transform_attribute.predict(attribute_new)
print(attribute_new, attribute_new.shape)


trainval_attributes = np.zeros((len(trainval_loc), attribute_new.shape[1]))
for i in range(len(trainval_loc)):
    trainval_attributes[i] = attribute_new[int(labels_trainval[i])-1]

print(trainval_attributes, trainval_attributes.shape)# (23527, 85)

87040
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 87040)]           0         
                                                                 
 layer1 (Dense)              (None, 4096)              356519936 
                                                                 
 layer3 (Dense)              (None, 2048)              8390656   
                                                                 
Total params: 364,910,592
Trainable params: 0
Non-trainable params: 364,910,592
_________________________________________________________________
2/2 [==============================] - 0s 5ms/step
[[-0.00382511  0.0777265  -0.02586024 ... -0.03016872  0.0993115
   0.00647516]
 [ 0.08858047  0.02310733  0.11753352 ... -0.07380248  0.07652213
  -0.05267316]
 [ 0.07576825 -0.00099139 -0.00923608 ... -0.15335062  0.11195775
  -0.06527486]
 ...
 [-0.05

In [11]:
trainval_input1 = np.reshape(trainval_vec, [trainval_vec.shape[0], 1, 1, trainval_vec.shape[1]])
print(trainval_input1.shape)

trainval_input2 = np.reshape(trainval_attributes, [trainval_attributes.shape[0], 1, 1, trainval_attributes.shape[1]])
print(trainval_input2.shape)

(23527, 1, 1, 2048)
(23527, 1, 1, 2048)


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence

X_train, X_val, y_train, y_val = train_test_split(trainval_input1, trainval_input2, test_size = 0.2, random_state = 42)

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 16
train_gen = DataGenerator(X_train, y_train, batch_size)   
val_gen = DataGenerator(X_val, y_val, batch_size)

In [13]:
# embedding model
# -----------------------------------------------------------------------------------
inputt = Input(shape = (1, 1, trainval_vec.shape[1]))
hidden1 = Conv2D(2048, 1, name="layer1", activation='relu')(inputt)
output1 = Conv2D(1024, 1, name="layer2", activation='relu')(hidden1)
hidden2 = Conv2D(1024, 1, name="layer3", activation='relu')(output1)
output2 = Conv2D(trainval_attributes.shape[1], 1, name="layer4", activation='linear')(hidden2)

embedding_model = Model(inputs = inputt, outputs = output2)

# embedding model variants
# -----------------------------------------------------------------------------------

#inputt = Input(shape = (1, 1, trainval_vec.shape[1]))
#hidden1 = Conv2D(1024, 1, name="layer1", activation='relu')(inputt)
#output1 = Conv2D(1024, 1, name="layer2", activation='relu')(hidden1)
#hidden2 = Conv2D(1024, 1, name="layer3", activation='relu')(output1)
#output2 = Conv2D(trainval_attributes.shape[1], 1, name="layer4", activation='linear')(hidden2)

#embedding_model = Model(inputs = inputt, outputs = output2)


adam = Adam(learning_rate = 0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
embedding_model.compile(adam, loss = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.AUTO), metrics = ['accuracy'])
#embedding_model.compile(adam, loss = cosine_loss, metrics = ['accuracy'])

embedding_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2D)             (None, 1, 1, 2048)        4196352   
                                                                 
 layer2 (Conv2D)             (None, 1, 1, 1024)        2098176   
                                                                 
 layer3 (Conv2D)             (None, 1, 1, 1024)        1049600   
                                                                 
 layer4 (Conv2D)             (None, 1, 1, 2048)        2099200   
                                                                 
Total params: 9,443,328
Trainable params: 9,443,328
Non-trainable params: 0
_________________________________________________________________


In [14]:

#CosineSimilarity is a number between -1 and 1. When it is a negative number between -1 and 0, 0 indicates orthogonality and 
#values closer to -1 indicate greater similarity. The values closer to 1 indicate greater dissimilarity. 

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_3/Results/AWA2/BertAttribute/with_LE/Embedding Model/'
name = 'embedding_model_Bert2_2048_1024_1024_AWA2_200eph_adam_cos_16bch_0.0001lr'

file_path = save_path + 'bw_' + name + '.h5'

#model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath = file_path,
#    monitor = 'val_accuracy',
#    mode = 'max',
#    save_best_only=True)

#train_summary = embedding_model.fit(train_gen, epochs = 200, verbose = 1, callbacks = [model_checkpoint_callback], validation_data = val_gen, 
#                              shuffle = True, steps_per_epoch = len(train_gen)//batch_size, 
#                              validation_steps = len(val_gen)//batch_size)

embedding_model.load_weights(save_path + 'bw_' + name + '.h5')

In [15]:
embedding_model_0 = Model(inputs = embedding_model.input, outputs = embedding_model.layers[-3].output) # for embedding model
#embedding_model_0 = Model(inputs = embedding_model.input, outputs = embedding_model.layers[-2].output) # for embedding model variants
embedding_model_0.summary()

trainval_vec = embedding_model_0.predict(trainval_input1)
print(trainval_vec, trainval_vec.shape)


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2D)             (None, 1, 1, 2048)        4196352   
                                                                 
 layer2 (Conv2D)             (None, 1, 1, 1024)        2098176   
                                                                 
Total params: 6,294,528
Trainable params: 6,294,528
Non-trainable params: 0
_________________________________________________________________
736/736 [==============================] - 1s 678us/step
[[[[0.         0.646958   0.5096992  ... 1.2153103  0.66170496
    0.        ]]]


 [[[0.06897884 0.         0.8608626  ... 2.8148031  2.0006714
    0.        ]]]


 [[[0.         0.         1.8988659  ... 2.3388317  2.5795538
    0.        ]]]


 ...


 [

In [ ]:
inputt = Input(shape = (1, 1, trainval_attributes.shape[1]))
hidden1 = Conv2DTranspose(1024, 1, name="layer1", activation='relu')(inputt)

embedding_model_1 = Model(inputs = inputt, outputs = hidden1)
embedding_model_1.summary()

w_list = Model(inputs = embedding_model_0.layers[-2].output, outputs = embedding_model_0.output).get_weights()
#print(weights_list2)

embedding_model_1.set_weights([np.transpose(w_list[0], (0, 1, 3, 2)), w_list[1]])

trainval_attributes = embedding_model_1.predict(trainval_input2)
print(trainval_attributes, trainval_attributes.shape)

In [16]:
## for LE new

#inputt = Input(shape = (1, 1, trainval_attributes.shape[1]))
#hidden1 = Conv2DTranspose(1024, 1, name="layer1", activation='relu')(inputt)
#hidden2 = Conv2DTranspose(1024, 1, name="layer2", activation='relu')(hidden1)

#embedding_model_1 = Model(inputs = inputt, outputs = hidden1)
#embedding_model_1 = Model(inputs = inputt, outputs = hidden2)
#embedding_model_1.summary()

#w_list = Model(inputs = embedding_model.layers[-3].output, outputs = embedding_model.output).get_weights()

#embedding_model_1.layers[-1].set_weights([np.transpose(w_list[0], (0, 1, 3, 2)), w_list[1]])
#embedding_model_1.layers[-2].set_weights([w_list[2], w_list[1]])


#trainval_attributes = embedding_model_1.predict(trainval_input2)
#print(trainval_attributes, trainval_attributes.shape)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2DTranspose)    (None, 1, 1, 1024)        2098176   
                                                                 
 layer2 (Conv2DTranspose)    (None, 1, 1, 1024)        1049600   
                                                                 
Total params: 3,147,776
Trainable params: 3,147,776
Non-trainable params: 0
_________________________________________________________________
736/736 [==============================] - 1s 690us/step
[[[[0.         0.         0.08818287 ... 0.         0.04905149
    0.        ]]]


 [[[0.         0.         0.08435377 ... 0.         0.0668256
    0.        ]]]


 [[[0.         0.         0.08818287 ... 0.         0.04905149
    0.        ]]]


 ...


 

In [17]:

attribute_new = np.reshape(attribute_new, [attribute_new.shape[0], 1, 1, attribute_new.shape[1]])

# as labels range from 1 to 50, we have subtract 1
attribute_2 = embedding_model_1.predict(attribute_new)
print(attribute_2, attribute_2.shape)


2/2 [==============================] - 0s 2ms/step
[[[[0.         0.         0.09267049 ... 0.         0.13768362
    0.        ]]]


 [[[0.03586135 0.02566641 0.08643061 ... 0.         0.0162082
    0.        ]]]


 [[[0.         0.         0.11268637 ... 0.03148045 0.
    0.        ]]]


 ...


 [[[0.00930722 0.         0.06534882 ... 0.         0.0441683
    0.        ]]]


 [[[0.         0.         0.14253864 ... 0.         0.06400539
    0.0064539 ]]]


 [[[0.         0.         0.13228218 ... 0.         0.
    0.        ]]]] (50, 1, 1, 1024)


In [18]:


signature_2 = attribute_2.transpose()
signature_2 = np.reshape(signature_2, [signature_2.shape[0], signature_2.shape[3]])
print(signature_2, signature_2.shape)#(50, 300)

trainval_sig = signature_2[:, (unique_labels_trainval)-1]
print("Signature for trainval:", trainval_sig.shape)


[[0.         0.03586135 0.         ... 0.00930722 0.         0.        ]
 [0.         0.02566641 0.         ... 0.         0.         0.        ]
 [0.09267049 0.08643061 0.11268637 ... 0.06534882 0.14253864 0.13228218]
 ...
 [0.         0.         0.03148045 ... 0.         0.         0.        ]
 [0.13768362 0.0162082  0.         ... 0.0441683  0.06400539 0.        ]
 [0.         0.         0.         ... 0.         0.0064539  0.        ]] (1024, 50)
Signature for trainval: (1024, 40)


In [19]:
# by doing this modification, we are changing the range of trainval and test seen labels from 0 to 39 
# and test unseen labels from 0 to 9

k = 0
new_labels_trainval = np.zeros((len(labels_trainval), 1), dtype = 'int')
for labels in unique_labels_trainval:
    new_labels_trainval[labels_trainval == labels] = k
    k = k+1
    
print(new_labels_trainval, new_labels_trainval.shape)#(23527, 1)
print(np.unique(new_labels_trainval), np.unique(new_labels_trainval).shape)


[[34]
 [19]
 [34]
 ...
 [32]
 [16]
 [37]] (23527, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] (40,)


In [20]:
#params for trainval and test set
m_trainval = new_labels_trainval.shape[0]# number of instances in training set: 23527
print(m_trainval)

z_trainval = len(unique_labels_trainval)# number of classes in training set: 40
print(z_trainval)


23527
40


In [21]:
from tensorflow.keras.utils import to_categorical
gt_trainval = to_categorical(new_labels_trainval, z_trainval)

print(gt_trainval, gt_trainval.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]] (23527, 40)


In [22]:
input1_shape = trainval_vec.shape[3]
print(input1_shape)

attribute_shape = trainval_sig.shape[0]
print(attribute_shape)

output_shape = z_trainval
print(output_shape)


1024
1024
40


In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad

# define model2 for attribute to class label mapping

input2 = Input(shape = (1, 1, attribute_shape))
flat = Flatten()(input2)
output = Dense(output_shape, name="output", activation='softmax')(flat)

model2 = Model(inputs = input2, outputs = output)


opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
#opt = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
model2.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model2.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1, 1, 1024)]      0         
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 output (Dense)              (None, 40)                41000     
                                                                 
Total params: 41,000
Trainable params: 41,000
Non-trainable params: 0
_________________________________________________________________


In [24]:
# define model1 for resnet feature to class label mapping

input1 = Input(shape = (1,1, input1_shape))
#inter_pre = Dense(512, name="intermediate_previous", activation='relu')(input1)
inter = Conv1D(attribute_shape, kernel_size = 1, name = "intermediate", activation = 'linear')(input1)
flat = Flatten()(inter)
output = Dense(output_shape, name="output", activation='softmax')(flat)

model1 = Model(inputs = input1, outputs = output)

opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
#opt = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
model1.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model1.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1, 1, 1024)]      0         
                                                                 
 intermediate (Conv1D)       (None, 1, 1, 1024)        1049600   
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 output (Dense)              (None, 40)                41000     
                                                                 
Total params: 1,090,600
Trainable params: 1,090,600
Non-trainable params: 0
_________________________________________________________________


In [25]:
trainval_input1 = trainval_vec
print(trainval_input1.shape)

trainval_input2 = trainval_attributes
print(trainval_input2.shape)

trainval_output = gt_trainval
print(trainval_output.shape)

(23527, 1, 1, 1024)
(23527, 1, 1, 1024)
(23527, 40)


In [26]:
from tensorflow.keras.utils import Sequence
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 16
from sklearn.model_selection import train_test_split    


X_train1, X_val1, y_train1, y_val1 = train_test_split(trainval_input1, trainval_output, test_size = 0.2, random_state = 42)

#train_gen1 = DataGenerator(X_train1, y_train1, batch_size)   
#val_gen1 = DataGenerator(X_val1, y_val1, batch_size)


X_train2, X_val2, y_train2, y_val2 = train_test_split(trainval_input2, trainval_output, test_size = 0.2, random_state = 42)

#train_gen2 = DataGenerator(X_train2, y_train2, batch_size)   
#val_gen2 = DataGenerator(X_val2, y_val2, batch_size)

In [27]:
labels = res101['labels']# direct class labels
print('labels', labels, labels.shape)# 37322 x 1

test_seen_loc = np.squeeze(att_splits['test_seen_loc']-1)
print(np.unique(test_seen_loc), np.max(np.unique(test_seen_loc))) # smallest location: 0, largest location 37319

labels_test_seen = labels[test_seen_loc]
print('labels_test_seen', labels_test_seen, labels_test_seen.shape)

unique_labels_test_seen = np.unique(labels_test_seen) # labels min:1 max:49
print('unique_labels_test_seen', unique_labels_test_seen, unique_labels_test_seen.shape)# 40 classes

test_seen_vec = X_features[:, test_seen_loc].transpose()
print("Features for test seen:", test_seen_vec.shape)# (5882, 2048)

test_seen_vec = np.reshape(test_seen_vec, [test_seen_vec.shape[0], 1, 1, test_seen_vec.shape[1]])
test_seen_vec = embedding_model_0.predict(test_seen_vec)
print(test_seen_vec.shape)


test_seen_attributes = np.zeros((len(test_seen_loc), attribute_2.shape[1], attribute_2.shape[2], attribute_2.shape[3]))
for i in range(len(test_seen_loc)):
    test_seen_attributes[i] = attribute_2[int(labels_test_seen[i])-1]

print(test_seen_attributes, test_seen_attributes.shape)# (5882, 85)



test_seen_sig = signature_2[:, (unique_labels_test_seen)-1]
print("Signature for test seen:", test_seen_sig.shape)

labels [[ 1]
 [ 1]
 [ 1]
 ...
 [38]
 [38]
 [38]] (37322, 1)
[    0     1     3 ... 37306 37307 37319] 37319
labels_test_seen [[22]
 [49]
 [14]
 ...
 [25]
 [15]
 [27]] (5882, 1)
unique_labels_test_seen [ 1  2  3  4  5  6  8 10 11 12 13 14 15 16 17 18 19 20 21 22 25 26 27 28
 29 32 33 35 36 37 38 39 40 42 43 44 45 46 48 49] (40,)
Features for test seen: (5882, 2048)
184/184 [==============================] - 0s 690us/step
(5882, 1, 1, 1024)
[[[[0.         0.         0.08435377 ... 0.         0.0668256
    0.        ]]]


 [[[0.         0.         0.14253864 ... 0.         0.06400539
    0.0064539 ]]]


 [[[0.         0.02559936 0.12289    ... 0.00933561 0.07491595
    0.        ]]]


 ...


 [[[0.         0.         0.02211225 ... 0.         0.02683282
    0.        ]]]


 [[[0.         0.         0.10873226 ... 0.         0.04536447
    0.        ]]]


 [[[0.00945213 0.         0.01766315 ... 0.         0.06192227
    0.        ]]]] (5882, 1, 1, 1024)
Signature for test seen: (1024, 40)

In [28]:

test_unseen_loc = np.squeeze(att_splits['test_unseen_loc']-1)
print(np.unique(test_unseen_loc), np.max(np.unique(test_unseen_loc))) # smallest location: 1046, largest location 35290

labels_test_unseen = labels[test_unseen_loc]
print('labels_test_unseen', labels_test_unseen, labels_test_unseen.shape)

unique_labels_test_unseen = np.unique(labels_test_unseen) # labels min:7 max:50
print('unique_labels_test_unseen', unique_labels_test_unseen, unique_labels_test_unseen.shape)# 10 classes


test_unseen_vec = X_features[:, test_unseen_loc].transpose()
print("Features for test unseen:", test_unseen_vec.shape) #(7913, 2048)


test_unseen_vec = np.reshape(test_unseen_vec, [test_unseen_vec.shape[0], 1, 1, test_unseen_vec.shape[1]])
test_unseen_vec = embedding_model_0.predict(test_unseen_vec)
print(test_unseen_vec.shape)

#test_unseen_attributes = np.zeros((len(test_unseen_loc), attribute_new.shape[1]))
#for i in range(len(test_unseen_loc)):
#    test_unseen_attributes[i] = attribute_new[int(labels_test_unseen[i])-1]

#print(test_unseen_attributes, test_unseen_attributes.shape)# (7913, 85)


#test_unseen_attributes_2 = model0.predict(test_unseen_attributes)
#print(test_unseen_attributes_2, test_unseen_attributes_2.shape)

test_unseen_sig = signature_2[:, (unique_labels_test_unseen)-1]

print("Signature for test unseen:", test_unseen_sig.shape)

[ 1046  1047  1048 ... 35288 35289 35290] 35290
labels_test_unseen [[30]
 [30]
 [30]
 ...
 [47]
 [47]
 [47]] (7913, 1)
unique_labels_test_unseen [ 7  9 23 24 30 31 34 41 47 50] (10,)
Features for test unseen: (7913, 2048)
248/248 [==============================] - 0s 689us/step
(7913, 1, 1, 1024)
Signature for test unseen: (1024, 10)


In [29]:

l = 0
new_labels_test_seen = np.zeros((len(labels_test_seen), 1), dtype = 'int')
for labels in unique_labels_test_seen:
    new_labels_test_seen[labels_test_seen == labels] = l
    l = l+1
    
print(new_labels_test_seen, new_labels_test_seen.shape)# (5882, 1)



n_test_seen = new_labels_test_seen.shape[0]# 5882
print(n_test_seen)

z1_test_seen = len(unique_labels_test_seen)# 40
print(z1_test_seen)


m = 0
new_labels_test_unseen = np.zeros((len(labels_test_unseen), 1), dtype = 'int')
for labels in unique_labels_test_unseen:
    new_labels_test_unseen[labels_test_unseen == labels] = m
    m = m+1  

print(new_labels_test_unseen, new_labels_test_unseen.shape) #  (7913, 1)  

n_test_unseen = new_labels_test_unseen.shape[0]# 7913
print(n_test_unseen)

z1_test_unseen = len(unique_labels_test_unseen)# 10
print(z1_test_unseen)


[[19]
 [39]
 [11]
 ...
 [20]
 [12]
 [22]] (5882, 1)
5882
40
[[4]
 [4]
 [4]
 ...
 [8]
 [8]
 [8]] (7913, 1)
7913
10


In [30]:
iteration = 15
epochs1 = 200
epochs2 = 200

best_performance_micro = [0, 0, 0]
best_performance_macro = [0, 0, 0]

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_3/Results/AWA2/BertAttribute/with_LE/'
name = 'model1_conv_AWA2_LEnew_Bert2attT_1024_lin_it15_200eph_adam_cce_16bch_1e-2lr_model2_adam_lr-2_200'

for i in range(iteration):
    X_train2_it = X_train2[(len(X_train2)//iteration)*i:(len(X_train2)//iteration)*(i+1)]
    X_val2_it = X_val2[(len(X_val2)//iteration)*i:(len(X_val2)//iteration)*(i+1)]
    y_train2_it = y_train2[(len(y_train2)//iteration)*i:(len(y_train2)//iteration)*(i+1)]
    y_val2_it = y_val2[(len(y_val2)//iteration)*i:(len(y_val2)//iteration)*(i+1)]
    
    train_gen2 = DataGenerator(X_train2_it, y_train2_it, batch_size)   
    val_gen2 = DataGenerator(X_val2_it, y_val2_it, batch_size)

    train_summary2 = model2.fit(train_gen2, epochs = epochs2, verbose = 0, callbacks = None, validation_data = val_gen2, 
                              shuffle = True, steps_per_epoch = len(train_gen2)//batch_size, 
                              validation_steps = len(val_gen2)//batch_size)

    print("iteration:", i)
    print('model 2 is trained:', 'training acc:', train_summary2.history['accuracy'][-1], ',',  
          'training loss:', train_summary2.history['loss'][-1], ',', 
          'validation acc:', train_summary2.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary2.history['val_loss'][-1])

    weights_list2 = model2.get_weights()
    #print(weights_list2)

    model1.layers[-1].set_weights(weights_list2)

    X_train1_it = X_train1[(len(X_train1)//iteration)*i:(len(X_train1)//iteration)*(i+1)]
    X_val1_it = X_val1[(len(X_val1)//iteration)*i:(len(X_val1)//iteration)*(i+1)]
    y_train1_it = y_train1[(len(y_train1)//iteration)*i:(len(y_train1)//iteration)*(i+1)]
    y_val1_it = y_val1[(len(y_val1)//iteration)*i:(len(y_val1)//iteration)*(i+1)]
    
    train_gen1 = DataGenerator(X_train1_it, y_train1_it, batch_size)   
    val_gen1 = DataGenerator(X_val1_it, y_val1_it, batch_size)
    
    for layer in model1.layers[2:]:
        layer.trainable = True

    train_summary1 = model1.fit(train_gen1, epochs = epochs1, verbose = 0, callbacks = None, validation_data = val_gen1, 
                              shuffle = True, steps_per_epoch = len(train_gen1)//batch_size, 
                              validation_steps = len(val_gen1)//batch_size)
    
    #print("iteration:", i)
    print('model 1 is trained:', 'training acc:', train_summary1.history['accuracy'][-1], ',',  
          'training loss:', train_summary1.history['loss'][-1], ',', 
          'validation acc:', train_summary1.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary1.history['val_loss'][-1])
    
    weights_list1 = Model(inputs = model1.input, outputs = model1.layers[-1].output).get_weights()
    
    #predictions
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, np.matmul(weights_list1[0], weights_list1[2])), test_seen_sig)
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0]), test_seen_sig)
    
    outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0][0]), test_seen_sig)
    preds_seen = np.array([np.argmax(output) for output in outputs_seen])
    
    cm_seen = confusion_matrix(new_labels_test_seen, preds_seen)
    #print(cm)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_seen_micro = cm_seen.astype('float') / cm_seen.sum(axis=1)[:, np.newaxis]
    #print(cm)
    avg_seen_micro = (sum(cm_seen_micro.diagonal())/len(unique_labels_test_seen))*100

    avg_seen_macro = (sum(cm_seen.diagonal())/len(new_labels_test_seen))*100
    
    #predictions
    #outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0]), test_unseen_sig)
    outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0][0]), test_unseen_sig)
    
    preds_unseen = np.array([np.argmax(output) for output in outputs_unseen])
    
    cm_unseen = confusion_matrix(new_labels_test_unseen, preds_unseen)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_unseen_micro = cm_unseen.astype('float') / cm_unseen.sum(axis=1)[:, np.newaxis]
    avg_unseen_micro = (sum(cm_unseen_micro.diagonal())/len(unique_labels_test_unseen))*100

    avg_unseen_macro = (sum(cm_unseen.diagonal())/len(new_labels_test_unseen))*100
    
    harmonic_micro = (2*avg_seen_micro*avg_unseen_micro) / (avg_seen_micro + avg_unseen_micro)
    harmonic_macro = (2*avg_seen_macro*avg_unseen_macro) / (avg_seen_macro + avg_unseen_macro)
    
    print('micro average')
    print('seen accuracy:', avg_seen_micro, 'unseen accuracy:', avg_unseen_micro, 'harmonic mean:', harmonic_micro)
    
    print('macro average')
    print('seen accuracy:', avg_seen_macro, 'unseen accuracy:', avg_unseen_macro, 'harmonic mean:', harmonic_macro)
    
    if harmonic_micro > best_performance_micro[2]:
        best_performance_micro = [avg_seen_micro, avg_unseen_micro, harmonic_micro]
        model1.save_weights(save_path + 'bw_micro_' + name + '.h5', overwrite=True)
        
    if harmonic_macro > best_performance_macro[2]:
        best_performance_macro = [avg_seen_macro, avg_unseen_macro, harmonic_macro]
        model1.save_weights(save_path + 'bw_macro_' + name + '.h5', overwrite=True)
        
    print('best accuracy micro','seen accuracy:', best_performance_micro[0], 'unseen accuracy:', best_performance_micro[1], 'harmonic mean:', best_performance_micro[2])
    print('best accuracy macro', 'seen accuracy:', best_performance_macro[0], 'unseen accuracy:', best_performance_macro[1], 'harmonic mean:', best_performance_macro[2])
    
    print('-----------------------------------------------------------------------------------------------------------')
    weights_list3 = model1.get_weights()
    model2.set_weights(weights_list3[2:])

iteration: 0
model 2 is trained: training acc: 0.859375 , training loss: 1.03725004196167 , validation acc: 0.8125 , validation_loss: 1.0718791484832764
model 1 is trained: training acc: 0.890625 , training loss: 1.6108466386795044 , validation acc: 0.75 , validation_loss: 10.454130172729492
micro average
seen accuracy: 73.12769999829378 unseen accuracy: 51.55851321337048 harmonic mean: 60.47750412028247
macro average
seen accuracy: 81.91091465487929 unseen accuracy: 46.36673827878175 harmonic mean: 59.21439713186096
best accuracy micro seen accuracy: 73.12769999829378 unseen accuracy: 51.55851321337048 harmonic mean: 60.47750412028247
best accuracy macro seen accuracy: 81.91091465487929 unseen accuracy: 46.36673827878175 harmonic mean: 59.21439713186096
-----------------------------------------------------------------------------------------------------------
iteration: 1
model 2 is trained: training acc: 1.0 , training loss: 0.3871808648109436 , validation acc: 1.0 , validation_loss:

iteration: 10
model 2 is trained: training acc: 1.0 , training loss: 0.10259518772363663 , validation acc: 1.0 , validation_loss: 0.12553998827934265
model 1 is trained: training acc: 0.953125 , training loss: 4.596445083618164 , validation acc: 0.9375 , validation_loss: 30.583168029785156
micro average
seen accuracy: 82.60267350078074 unseen accuracy: 43.07422612175757 harmonic mean: 56.622119853699154
macro average
seen accuracy: 87.87827269636178 unseen accuracy: 38.607354985466955 harmonic mean: 53.64637440124778
best accuracy micro seen accuracy: 81.58759177767548 unseen accuracy: 48.676113581224534 harmonic mean: 60.97426559833962
best accuracy macro seen accuracy: 81.91091465487929 unseen accuracy: 46.36673827878175 harmonic mean: 59.21439713186096
-----------------------------------------------------------------------------------------------------------
iteration: 11
model 2 is trained: training acc: 1.0 , training loss: 0.10797490179538727 , validation acc: 1.0 , validation_lo

In [31]:
gt_test_seen = to_categorical(new_labels_test_seen, z1_test_seen)

print(gt_test_seen, gt_test_seen.shape)

#test_seen_vec = np.reshape(test_seen_vec, [test_seen_vec.shape[0], 1, 1, test_seen_vec.shape[1]])
res1 = model1.evaluate(test_seen_vec, gt_test_seen)

p1 = model1.predict(test_seen_vec, verbose = 0)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
print('cce = ', cce(gt_test_seen, p1).numpy())

#test_seen_attributes = np.reshape(train_attributes, [test_seen_attributes.shape[0], 1, train_attributes.shape[1]])
#test_seen_attributes = np.reshape(test_seen_attributes, [test_seen_attributes.shape[0], 1, 1, test_seen_attributes.shape[1]])

res2 = model2.evaluate(test_seen_attributes, gt_test_seen)

p2 = model2.predict(test_seen_attributes, verbose = 0)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
print('cce = ', cce(gt_test_seen, p2).numpy())


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (5882, 40)
184/184 [==============================] - 0s 2ms/step - loss: 29.5567 - accuracy: 0.9033
cce =  1.5199095
184/184 [==============================] - 0s 1ms/step - loss: 0.1369 - accuracy: 1.0000
cce =  0.13685596


In [32]:

accuracy_seen_updated = res1[1]*100
unseen_accuracy = 59.54
h = (2*accuracy_seen_updated*unseen_accuracy) / (accuracy_seen_updated + unseen_accuracy)
print(h)


accuracy_seen_updated2 = ((res1[1]*100)+(res2[1]*100))/2
print(accuracy_seen_updated2)
h = (2*accuracy_seen_updated2*unseen_accuracy) / (accuracy_seen_updated2 + unseen_accuracy)
print(h)

71.77104924268816
95.16321122646332
73.25016140911761


In [33]:
from sklearn.metrics import precision_recall_fscore_support

pp1 = np.array([np.argmax(output) for output in p1])
pp2 = np.array([np.argmax(output) for output in p2])

seen_macro1 = precision_recall_fscore_support(new_labels_test_seen, pp1, average = 'macro')
seen_macro2 = precision_recall_fscore_support(new_labels_test_seen, pp2, average = 'macro')
print('precision_seen_macro', (seen_macro1[0] + seen_macro2[0])/2, 'recall_seen_macro', (seen_macro1[1] + seen_macro2[1])/2, 'f1_seen_macro', (seen_macro1[2] + seen_macro2[2])/2)


seen_micro1 = precision_recall_fscore_support(new_labels_test_seen, pp1, average = 'micro')
seen_micro2 = precision_recall_fscore_support(new_labels_test_seen, pp2, average = 'micro')
print('precision_seen_micro', (seen_micro1[0] + seen_micro2[0])/2, 'recall_seen_micro', (seen_micro1[1] + seen_micro2[1])/2, 'f1_seen_micro', (seen_micro1[2] + seen_micro2[2])/2)

unseen_macro = precision_recall_fscore_support(new_labels_test_unseen, preds_unseen, average = 'macro')
unseen_micro = precision_recall_fscore_support(new_labels_test_unseen, preds_unseen, average = 'micro')

print('precision_unseen_macro', unseen_macro[0], 'recall_unseen_macro', unseen_macro[1], 'f1_unseen_macro', unseen_macro[2])
print('precision_unseen_micro', unseen_micro[0], 'recall_unseen_micro', unseen_micro[1], 'f1_unseen_micro', unseen_micro[2])


precision_seen_macro 0.9522035396323645 recall_seen_macro 0.9365450051607678 f1_seen_macro 0.9413005107028479
precision_seen_micro 0.9516320979258756 recall_seen_micro 0.9516320979258756 f1_seen_micro 0.9516320979258756
precision_unseen_macro 0.46990354969739967 recall_unseen_macro 0.4455030541043222 f1_unseen_macro 0.36606799687401165
precision_unseen_micro 0.42626058384936183 recall_unseen_micro 0.42626058384936183 f1_unseen_micro 0.42626058384936183
